# LOAD SILVER LAYER (BRONZE → SILVER)

---

## Purpose
This notebook (or procedure) performs the **ETL (Extract, Transform, Load)** process to populate the **silver** schema tables from the **bronze** schema.

---

## Actions Performed
- Set datestyle (if needed in SQL environments)
- Truncate (overwrite) the target silver tables before loading
- Insert **transformed and cleaned** data from bronze into silver tables

---

## Parameters
- Paths to **Bronze-layer Delta tables** (input)
- Target **Silver Delta location**

---

## Usage Example
- Databricks interactive notebook.

In [0]:
from datetime import datetime
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, TimestampType
from pyspark.sql.functions import to_date, to_timestamp

In [0]:
# path
bronze_path = 'gs://my-bucket-deep/Medallion/bronze/'
silver_path = 'gs://my-bucket-deep/Medallion/silver/'

In [0]:
bronze_tables = [
    "crm_cust_info",
    "crm_prd_info",
    "crm_sales_details",
    "erp_cust_az12",
    "erp_loc_a101",
    "erp_px_cat_g1v2"
]

bronze_dfs = {
    table: spark.read.format('delta').load(f'{bronze_path}{table}')
    for table in bronze_tables
}

In [0]:
bronze_dfs

{'crm_cust_info': DataFrame[cst_id: int, cst_key: string, cst_firstname: string, cst_lastname: string, cst_marital_status: string, cst_gndr: string, cst_create_date: date],
 'crm_prd_info': DataFrame[prd_id: int, prd_key: string, prd_nm: string, prd_cost: int, prd_line: string, prd_start_dt: timestamp, prd_end_dt: timestamp],
 'crm_sales_details': DataFrame[sls_ord_num: string, sls_prd_key: string, sls_cust_id: int, sls_order_dt: int, sls_ship_dt: int, sls_due_dt: int, sls_sales: int, sls_quantity: int, sls_price: int],
 'erp_cust_az12': DataFrame[cid: string, bdate: date, gen: string],
 'erp_loc_a101': DataFrame[cid: string, cntry: string],
 'erp_px_cat_g1v2': DataFrame[id: string, cat: string, subcat: string, maintenance: string]}

In [0]:
# Silver Source
silver_sources = []

In [0]:
# Utility logger
def log(msg):
    print(f"[{datetime.now().isoformat()}] {msg}")

Transformation `crm_cust_info`

In [0]:
from pyspark.sql import functions as f
from pyspark.sql.window import Window

In [0]:
df = bronze_dfs['crm_cust_info']

# Trim columns
for col in ['cst_firstname', 'cst_lastname']:
    df = df.withColumn(col, f.trim(f.col(col)))

# Map marital status
df = df.withColumn(
    'cst_marital_status',
    f.when(f.upper(f.trim(f.col('cst_marital_status'))) == 'S', 'Single')
     .when(f.upper(f.trim(f.col('cst_marital_status'))) == 'M', 'Married')
     .otherwise('n/a')
)

# Map gender
df = df.withColumn(
    'cst_gndr',
    f.when(f.upper(f.trim(f.col('cst_gndr'))) == 'M', 'Male')
     .when(f.upper(f.trim(f.col('cst_gndr'))) == 'F', 'Female')
     .otherwise('n/a')
)

# Deduplication
window_spec = Window.partitionBy('cst_id').orderBy(f.col('cst_create_date').desc())
df = df.withColumn('row_num', f.row_number().over(window_spec)).filter(f.col('row_num') == 1).drop('row_num')

In [0]:
# Add to silver sources
silver_sources.append({
        'name': 'crm_cust_info',
        'path': f"{silver_path}crm_cust_info",
        'df': df
    })

Transformation `crm_prd_info`

In [0]:
df = bronze_dfs['crm_prd_info']

# Cast the timestamp
df = df.withColumn('prd_start_dt', f.to_date('prd_start_dt'))
df = df.withColumn('prd_end_dt', f.to_date('prd_end_dt'))

# Extract cat_id and prd_key
df = df.withColumn('cat_id', f.regexp_replace(f.substring('prd_key', 1, 5), '-', '_'))
df = df.withColumn('prd_key', f.substring('prd_key', 7, f.length('prd_key')))

# # Null cost to 0
df = df.withColumn('prd_cost', f.coalesce(f.col('prd_cost'), f.lit(0)))

# Map Product line codes to descriptive values
df = df.withColumn(
    'prd_line',
    f.when(f.upper(f.trim(f.col('prd_line'))) == 'M', 'Mountain')
     .when(f.upper(f.trim(f.col('prd_line'))) == 'R', 'Road')
     .when(f.upper(f.trim(f.col('prd_line'))) == 'S', 'Other Sales')
     .when(f.upper(f.trim(f.col('prd_line'))) == 'T', 'Touring')
     .otherwise('n/a')
)

# LEAD window for prd_end_dt
window_spec = Window.partitionBy('prd_key').orderBy('prd_start_dt')
df = df.withColumn(
    'prd_end_dt',
    f.lead('prd_start_dt').over(window_spec) - f.expr('INTERVAL 1 DAY')
)

df = df.select(['prd_id', 'cat_id', 'prd_key', 'prd_nm', 'prd_cost', 'prd_line', 'prd_start_dt', 'prd_end_dt'])

# Add to silver sources
silver_sources.append(
    {
        'name': 'crm_prd_info',
        'path': f"{silver_path}crm_prd_info",
        'df': df
    }
)

Transforation `crm_sales_details`

In [0]:
df = bronze_dfs['crm_sales_details']

# Valid date
def valid_date(col):
    return f.when((f.col(col) > 0) & (f.length(f.col(col).cast('string')) == 8),
                  f.to_date(f.col(col).cast('string'), 'yyyyMMdd')).otherwise(None)
    
# Cast the timestamp
df = df.withColumn("sls_order_dt", valid_date("sls_order_dt"))
df = df.withColumn("sls_ship_dt", valid_date("sls_ship_dt"))
df = df.withColumn("sls_due_dt", valid_date("sls_due_dt"))

# Recalculated sales
df = df.withColumn(
    'sls_sales',
    f.when(
        (f.col('sls_sales').isNull()) | (f.col('sls_sales') <= 0) |
        (f.col('sls_sales') != f.col('sls_quantity') * f.abs(f.col('sls_price'))),
        f.col('sls_quantity') * f.abs(f.col('sls_price'))
    ).otherwise(f.col('sls_sales'))
)

# Drive price if invalid
df = df.withColumn(
    "sls_price",
    f.when(
        (f.col("sls_price").isNull()) | (f.col("sls_price") <= 0),
        f.abs(f.col("sls_sales") / f.when(f.col("sls_quantity") != 0, f.col("sls_quantity")).otherwise(1))
    ).otherwise(f.col("sls_price"))
)

# Add to silver sources
silver_sources.append(
    {
        'name': 'crm_sales_details',
        'path': f"{silver_path}crm_sales_details",
        'df': df
    }
)

Tansformation `erp_cust_az12`

In [0]:
df = bronze_dfs['erp_cust_az12']

# Remove 'NAS' prefix
df = df.withColumn(
    "cid",
    f.when(f.col("cid").startswith("NAS"), f.expr("substring(cid, 4)")).otherwise(f.col("cid"))
)

# Future birthdate → NULL
df = df.withColumn(
    "bdate",
    f.when(f.col("bdate") > f.current_date(), None).otherwise(f.col("bdate"))
)

# Normalize gender
df = df.withColumn(
    "gen",
    f.when(f.upper(f.trim(f.col("gen"))).isin("F", "FEMALE"), "Female")
     .when(f.upper(f.trim(f.col("gen"))).isin("M", "MALE"), "Male")
     .otherwise("n/a")
)

silver_sources.append({
    "name": "erp_cust_az12",
    "path": f"{silver_path}erp_cust_az12",
    "df": df
})

Transformation `erp_loc_a101`

In [0]:
df = bronze_dfs['erp_loc_a101']

# Remove '-' in cid
df = df.withColumn("cid", f.regexp_replace("cid", "-", ""))

# Normalize country
df = df.withColumn(
    "cntry",
    f.when(f.trim(f.col("cntry")) == "DE", "Germany")
     .when(f.trim(f.col("cntry")).isin("US", "USA"), "United States")
     .when((f.col("cntry").isNull()) | (f.trim(f.col("cntry")) == ""), "n/a")
     .otherwise(f.trim(f.col("cntry")))
)

silver_sources.append({
    "name": "erp_loc_a101",
    "path": f"{silver_path}/erp_loc_a101",
    "df": df
})

Tansformation `erp_px_cat_g1v2`

In [0]:
df = bronze_dfs['erp_px_cat_g1v2']

silver_sources.append({
    "name": "erp_px_cat_g1v2",
    "path": f"{silver_path}/erp_px_cat_g1v2",
    "df": df
})

In [0]:
def load_to_silver_table(source):
    try:
        log(f"START: Writing {source['name']} to {source['path']}")
        
        source['df'].write.format('delta').mode('overwrite').option('overwriteSchema', 'true').save(source['path'])
        # Basic check
        record_count = source['df'].count()
        log(f"SUCCESS: Written {source['name']} with {record_count} records")

        return {
            "table_name": source['name'],
            "status": "SUCCESS",
            "records": record_count,
            "error": "",
            "loaded_at": datetime.now().isoformat(),
            "target_path": source['path']
        }
    except Exception as e:
        log(f"ERROR: {source['name']} - {e}")
        return {
            "table_name": source['name'],
            "status": "FAILED",
            "records": 0,
            "error": str(e),
            "loaded_at": datetime.now().isoformat(),
            "target_path": source['path']
        }

In [0]:
# Define Audit Schema
audit_schema = StructType([
    StructField("table_name", StringType(), True),
    StructField("status", StringType(), True),
    StructField("records", IntegerType(), True),
    StructField("error", StringType(), True),
    StructField("loaded_at", StringType(), True),
    StructField("target_path", StringType(), True)
])

In [0]:
# Process all sources and collect audit records
audit_records = []
for source in silver_sources:
    result = load_to_silver_table(source)
    audit_records.append(result)

audit_df = spark.createDataFrame(audit_records, schema=audit_schema)

# Write audit logs
audit_log_path = f"{silver_path}_silver_audit_logs"
audit_df.write.format('delta').mode('append').save(audit_log_path)

log(f"SUCCESS: Audit logs written to {audit_log_path}")

[2025-07-05T09:49:01.993689] START: Writing crm_cust_info to gs://my-bucket-deep/Medallion/silver/crm_cust_info
[2025-07-05T09:49:08.826173] SUCCESS: Written crm_cust_info with 18485 records
[2025-07-05T09:49:08.826427] START: Writing crm_prd_info to gs://my-bucket-deep/Medallion/silver/crm_prd_info
[2025-07-05T09:49:15.363432] SUCCESS: Written crm_prd_info with 397 records
[2025-07-05T09:49:15.363620] START: Writing crm_sales_details to gs://my-bucket-deep/Medallion/silver/crm_sales_details
[2025-07-05T09:49:23.084628] SUCCESS: Written crm_sales_details with 60398 records
[2025-07-05T09:49:23.084808] START: Writing erp_cust_az12 to gs://my-bucket-deep/Medallion/silver/erp_cust_az12
[2025-07-05T09:49:30.615054] SUCCESS: Written erp_cust_az12 with 18484 records
[2025-07-05T09:49:30.615246] START: Writing erp_loc_a101 to gs://my-bucket-deep/Medallion/silver//erp_loc_a101
[2025-07-05T09:49:37.422917] SUCCESS: Written erp_loc_a101 with 18484 records
[2025-07-05T09:49:37.423104] START: Writ

In [0]:
audit_df.display()

table_name,status,records,error,loaded_at,target_path
crm_cust_info,SUCCESS,18485,,2025-07-05T09:49:08.826360,gs://my-bucket-deep/Medallion/silver/crm_cust_info
crm_prd_info,SUCCESS,397,,2025-07-05T09:49:15.363567,gs://my-bucket-deep/Medallion/silver/crm_prd_info
crm_sales_details,SUCCESS,60398,,2025-07-05T09:49:23.084762,gs://my-bucket-deep/Medallion/silver/crm_sales_details
erp_cust_az12,SUCCESS,18484,,2025-07-05T09:49:30.615201,gs://my-bucket-deep/Medallion/silver/erp_cust_az12
erp_loc_a101,SUCCESS,18484,,2025-07-05T09:49:37.423057,gs://my-bucket-deep/Medallion/silver//erp_loc_a101
erp_px_cat_g1v2,SUCCESS,37,,2025-07-05T09:49:45.673350,gs://my-bucket-deep/Medallion/silver//erp_px_cat_g1v2
